In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np
import re
import requests
import random
from tqdm import tqdm
from datetime import datetime
import os
from os import listdir
import glob
import pickle #for saving data
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("darkgrid")

#show all columns pandas
pd.set_option('display.max_columns', None)

In [3]:
ad_data_all = pd.read_pickle("data/ad_data.pkl")

In [4]:
ad_data_all.head()

,header_title,header_subtitle,ad_link,ad_download_date_time,Asking price,Asking price per m²,Listed since,Status,Acceptance,Kind of house,Building type,Year of construction,Accessibility,Specific,Type of roof,Areas,Living area,External storage space,Plot size,Volume in cubic meters,Number of rooms,Number of bath rooms,Bathroom facilities,Number of stories,Facilities,Energy label,Insulation,Heating,Hot water,Area,Ownership situation,Location,Garden,Front garden,Garden location,Shed / storage,Facilities.1,Insulation.1,Type of parking facilities,error,VVE (Owners Association) contribution,Type apartment,Exterior space attached to the building,Located at,Balcony/roof garden,Registration with KvK,Annual meeting,Periodic contribution,Reserve fund present,Maintenance plan,Building insurance,Service charges,Fees,Type of garage,Capacity,Other space inside the building,CH boiler,Facilities.2,Back garden,Provisional energy label,Construction period,Ownership situation.1,Quality marks,Original asking price,Sun terrace,Ownership situation.2,Ownership situation.3,Fees.1,Patio/atrium,Purchase combination,Aangeboden sinds,Verkoopdatum,Looptijd,Dimensions,Ownership situation.4,Type of property,Insulation.2,Side garden,Fees.2,Fees.3,Location.1,Commercial property,Consulting rooms,Office space,Retail space,Area.1,Type of other property,Ownership situation.5,Ownership situation.6,Price,Auction date,Fees.4,Fees.5,Fees.6,Ownership situation.7,Fees.7,Ownership situation.8,Fees.8,Ownership situation.9,Fees.9,Ownership situation.10,Fees.10
0,Ben Websterstraat 16,3069 XJ Rotterdam,https://www.funda.nl/en/koop/rotterdam/huis-42...,20220125_212533,"€ 400,000 kosten koper","€ 3,279 The asking price per m² (€ 400,000)...",7 weeks,Sold under reservation,Available in consultation,"Single-family home, corner house (kwadrant won...",Resale property,1987,Disabled access friendly and accessible for se...,Partly furnished with carpets and curtains,Gable roof covered with roof tiles,Living area122 m²External storage space15 m²,122 m²,15 m²,156 m²,399 m³,4 rooms (3 bedrooms),2 bathrooms,Bath and 2 toilets,3 stories and a loft,"Air conditioning, outdoor awning, skylight, me...",B What does this mean?,"Roof insulation, double glazing, insulated wal...",Fireplace and district heating,District heating,156 m²,Full ownership,Sheltered location and in residential district,Front garden and side garden,72 m² (6 meter deep and 12 meter broad),Located at the west,Detached wooden storage,Electricity,No insulation,Public parking,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Laagjes 11,3076 BJ Rotterdam,https://www.funda.nl/en/koop/rotterdam/huis-88...,20220125_212558,"€ 650,000 kosten koper","€ 3,801 The asking price per m² (€ 650,000)...",5 months,Available,Available in consultation,"Single-family home, row house (official/compan...",Resale property,1982,NaN,NaN,Flat roof covered with asphalt roofing,Living area171 m²Other space inside the buildi...,171 m²,NaN,282 m²,"1,640 m³",5 rooms (3 bedrooms),2 separate toilets,NaN,2 stories,Rolldown shutters and solar panels,Not available,Mostly double glazed,CH boiler,CH boiler and gas-fired boiler,282 m²,Full ownership,In residential district,NaN,NaN,NaN,Built-in,Loft and electricity,NaN,Parking on private property and public parking,no,NaN,NaN,78 m²,NaN,Roof terrace present,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Built-in,6 cars,229 m²,"HR ( combination boiler from 2017, in ownership)",Electrical door and electricity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bergweg 41 d,3037 EA Rotterdam,https://www.funda.nl/en/koop/rotterdam/apparte...,20220125_212607,"€ 525,000 kosten koper","€ 3,571 The asking price p

In [5]:
def clean_data(data):
    
    data['price'] = data['Asking price'].str.replace('.', '')
    data['price'] = data['price'].str.replace(',', '')
    data['price'] = data['price'].str.extract('(\d+)').astype(int, errors='ignore')

    data = data.loc[~data['price'].isna()]
    data['price'] = data['price'].astype(int)

    data['livingArea'] = data['Living area'].str.replace('.', '')
    data['livingArea'] = data['livingArea'].str.replace(',', '')
    data['livingArea'] = data['livingArea'].str.extract('(\d+)').astype(int, errors='ignore')

    data = data.loc[~data['livingArea'].isna()]
    data['livingArea'] = data['livingArea'].astype(int)

    data['sqm_price'] = data['price'] / data['livingArea']
    #round to 2 decimals
    data['sqm_price'] =  data['sqm_price'].round(0)

    #extract city from link
    data['city'] = data['ad_link'].str.split('koop/').str[-1]
    data['city'] = data['city'].str.split('/').str[0]
    data['city'] = data['city'].str.strip()

    data['h_type'] = data['ad_link'].str.split('koop/').str[-1]
    data['h_type'] = data['h_type'].str.split('/').str[1]

    data['h_type'] = pd.np.where(data.h_type.str.contains("appartement"), "appartement", pd.np.where(data.h_type.str.contains("huis"), "huis", "unknown"))
    
    #get first letter sting pandas column
    data['energy_label'] = data['Energy label'].str[0]

    data['postcode'] = data['header_subtitle'].str[0:7]
    data['City'] = data['header_subtitle'].str[7:]
    data['City'] = data['City'].str.strip()

    data['address'] = data['header_title']
    data['postcode_city'] = data['header_subtitle']

    #keep only selected columns
    data = data[['address', 'postcode_city', 'postcode', 'City', 'city', 'ad_link', 'ad_download_date_time', 'Asking price', 'Original asking price', 'price', 'sqm_price',
      'Living area', 'livingArea', 'Plot size', 'Area', 'Volume in cubic meters', 'h_type', 'Kind of house', 'Type apartment', 'Number of stories', 'Located at', 'Location',
      'Building type', 'Year of construction', 'Construction period', 'Number of rooms', 'Number of bath rooms', 'Bathroom facilities',
      'External storage space',  'Garden', 'Front garden', 'Back garden', 'Garden location', 'Balcony/roof garden', 'Sun terrace',
      'Shed / storage', 'Patio/atrium', 'Exterior space attached to the building', 'Type of parking facilities', 'Type of garage', 'Other space inside the building', 
      'Facilities',  'Facilities.2', 'Ownership situation', 'Accessibility', 'Specific', 'Type of roof',  'energy_label', 'Provisional energy label', 'Insulation', 'Heating','CH boiler', 'Hot water',
      'VVE (Owners Association) contribution', 'Registration with KvK', 'Annual meeting', 'Periodic contribution', 'Reserve fund present', 'Maintenance plan', 'Building insurance',
      'Service charges', 'Fees', 'Quality marks',  'Purchase combination', 'Commercial property', 'Consulting rooms', 'Office space', 'Retail space']]

   

    return(data)

In [8]:
#turn off warnings
import warnings
warnings.filterwarnings('ignore')

data_clean = clean_data(ad_data_all)

#turn on warnings
warnings.filterwarnings('default')

In [7]:
# save data to csv
data_clean.to_csv("data/data_clean.csv", index=False)

In [216]:
data_clean

,address,postcode_city,postcode,City,city,ad_link,ad_download_date_time,Asking price,Original asking price,price,sqm_price,Living area,livingArea,Plot size,Area,Volume in cubic meters,h_type,Kind of house,Type apartment,Number of stories,Located at,Location,Building type,Year of construction,Construction period,Number of rooms,Number of bath rooms,Bathroom facilities,External storage space,Garden,Front garden,Back garden,Garden location,Balcony/roof garden,Sun terrace,Shed / storage,Patio/atrium,Exterior space attached to the building,Type of parking facilities,Type of garage,Other space inside the building,Facilities,Facilities.2,Ownership situation,Accessibility,Specific,Type of roof,energy_label,Provisional energy label,Insulation,Heating,CH boiler,Hot water,VVE (Owners Association) contribution,Registration with KvK,Annual meeting,Periodic contribution,Reserve fund present,Maintenance plan,Building insurance,Service charges,Fees,Quality marks,Purchase combination,Commercial property,Consulting rooms,Office space,Retail space
0,Ben Websterstraat 16,3069 XJ Rotterdam,3069 XJ,Rotterdam,rotterdam,https://www.funda.nl/en/koop/rotterdam/huis-42...,20220125_212533,"€ 400,000 kosten koper",NaN,400000,3279.0,122 m²,122,156 m²,156 m²,399 m³,huis,"Single-family home, corner house (kwadrant won...",NaN,3 stories and a loft,NaN,Sheltered location and in residential district,Resale property,1987,NaN,4 rooms (3 bedrooms),2 bathrooms,Bath and 2 toilets,15 m²,Front garden and side garden,72 m² (6 meter deep and 12 meter broad),NaN,Located at the west,NaN,NaN,Detached wooden storage,NaN,NaN,Public parking,NaN,NaN,"Air conditioning, outdoor awning, skylight, me...",NaN,Full ownership,Disabled access friendly and accessible for se...,Partly furnished with carpets and curtains,Gable roof covered with roof tiles,B,NaN,"Roof insulation, double glazing, insulated wal...",Fireplace and district heating,NaN,District heating,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Laagjes 11,3076 BJ Rotterdam,3076 BJ,Rotterdam,rotterdam,https://www.funda.nl/en/koop/rotterdam/huis-88...,20220125_212558,"€ 650,000 kosten koper",NaN,650000,3801.0,171 m²,171,282 m²,282 m²,"1,640 m³",huis,"Single-family home, row house (official/compan...",NaN,2 stories,NaN,In residential district,Resale property,1982,NaN,5 rooms (3 bedrooms),2 separate toilets,NaN,NaN,NaN,NaN,NaN,NaN,Roof terrace present,NaN,Built-in,NaN,78 m²,Parking on private property and public parking,Built-in,229 m²,Rolldown shutters and solar panels,Electrical door and electricity,Full ownership,NaN,NaN,Flat roof covered with asphalt roofing,N,NaN,Mostly double glazed,CH boiler,"HR ( combination boiler from 2017, in ownership)",CH boiler and gas-fired boiler,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bergweg 41 d,3037 EA Rotterdam,3037 EA,Rotterdam,rotterdam,https://www.funda.nl/en/koop/rotterdam/apparte...,20220125_212607,"€ 525,000 kosten koper",NaN,525000,3571.0,147 m²,147,NaN,NaN,495 m³,appartement,NaN,Ground-floor apartment (apartment),2 stories,Ground floor,NaN,Resale property,1935,NaN,5 rooms (3 bedrooms),NaN,NaN,NaN,Back garden,NaN,"19 m² (3,35 meter deep and 5,6 meter broad)",Located at the west,NaN,NaN,NaN,NaN,NaN,Paid parking and public parking,NaN,NaN,NaN,NaN,Full ownership,NaN,NaN,NaN,A,NaN,Double glazing,CH boiler,"Gas-fired combination boiler from 2021, in own...",CH boiler,NaN,Yes,No,Yes,Yes,No,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Molensingel 94,3043 KC Rotterdam,3043 KC,Rotterdam,rotterdam,https://www.funda.nl/en/koop/rotterdam/apparte...,20220125_212616,"€ 295,000 kosten koper",NaN,295000,3598.0,82 m²,82,NaN,NaN,320 m³,appartement,NaN,Upstairs apartment (apartment),3 stories,1st floor,"Alongside park, alongside a quiet road, in res...",Resale property,1938,NaN,4 rooms (3 bedrooms),1 bathroom and 1 separate toilet,Shower,4 m²,NaN,NaN,NaN,NaN,NaN,NaN,Detached wooden storage,NaN,3 m²,Public parking,NaN,10 m²,NaN,NaN,Full ownership,NaN,NaN,Gable roof covered